In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

# 创建一个包含缺失值的数据集
np.random.seed(42)
n_samples = 1000
data = {
    'feature1': np.random.rand(n_samples),
    'feature2': np.random.rand(n_samples),
    'feature3': np.random.rand(n_samples),
    'label': np.random.randint(0, 2, n_samples)
}
df = pd.DataFrame(data)

# 引入一些缺失值
for feature in ['feature1', 'feature2', 'feature3']:
    df.loc[np.random.choice(df.index, 100), feature] = np.nan

print("数据集中的缺失值:")
print(df.isnull().sum())

# 分离特征和标签
X = df[['feature1', 'feature2', 'feature3']]
y = df['label']

# 分割训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 定义不同的缺失值处理方法
imputers = {
    '均值填充': SimpleImputer(strategy='mean'),
    '中位数填充': SimpleImputer(strategy='median'),
    'KNN填充': KNNImputer(n_neighbors=5),
    '迭代填充': IterativeImputer(random_state=42, estimator=RandomForestRegressor())
}

# 创建"缺失"标志
for feature in X.columns:
    X_train[f'{feature}_is_missing'] = X_train[feature].isnull().astype(int)
    X_test[f'{feature}_is_missing'] = X_test[feature].isnull().astype(int)

# 比较不同方法的性能
for name, imputer in imputers.items():
    # 填充缺失值
    X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
    X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)
    
    # 特征缩放
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_imputed)
    X_test_scaled = scaler.transform(X_test_imputed)
    
    # 训练和评估模型
    mlp = MLPClassifier(max_iter=1000, random_state=42)
    scores = cross_val_score(mlp, X_train_scaled, y_train, cv=5)
    
    print(f"\n{name}:")
    print(f"交叉验证平均准确率: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")
    
    mlp.fit(X_train_scaled, y_train)
    train_accuracy = mlp.score(X_train_scaled, y_train)
    test_accuracy = mlp.score(X_test_scaled, y_test)
    print(f"训练集准确率: {train_accuracy:.4f}")
    print(f"测试集准确率: {test_accuracy:.4f}")

数据集中的缺失值:
feature1    96
feature2    93
feature3    95
label        0
dtype: int64

均值填充:
交叉验证平均准确率: 0.5137 (+/- 0.0696)
训练集准确率: 0.6162
测试集准确率: 0.4850

中位数填充:
交叉验证平均准确率: 0.5150 (+/- 0.0610)
训练集准确率: 0.6150
测试集准确率: 0.4950

KNN填充:
交叉验证平均准确率: 0.5300 (+/- 0.0366)
训练集准确率: 0.6525
测试集准确率: 0.5000


C:\Users\Public\anaconda3\Lib\site-packages\sklearn\impute\_iterative.py:801: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(



迭代填充:
交叉验证平均准确率: 0.5188 (+/- 0.0481)
训练集准确率: 0.6200
测试集准确率: 0.5000


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer

# 创建一个包含缺失值的数据集
np.random.seed(42)
n_samples = 1000
data = {
    'feature1': np.random.rand(n_samples),
    'feature2': np.random.rand(n_samples),
    'label': np.random.randint(0, 2, n_samples)
}
df = pd.DataFrame(data)

# 引入一些缺失值
df.loc[np.random.choice(df.index, 100), 'feature1'] = np.nan
df.loc[np.random.choice(df.index, 100), 'feature2'] = np.nan

print("数据集中的缺失值:")
print(df.isnull().sum())

# 分离特征和标签
X = df[['feature1', 'feature2']]
y = df['label']

# 分割训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 尝试直接使用带有缺失值的数据训练BP神经网络
try:
    mlp = MLPClassifier(max_iter=1000, random_state=42)
    mlp.fit(X_train, y_train)
except Exception as e:
    print("\nBP神经网络无法直接处理缺失值:")
    print(str(e))

# 处理缺失值
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

print("\n缺失值已被均值填充")

# 特征缩放
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# 使用处理后的数据训练BP神经网络
mlp = MLPClassifier(max_iter=1000, random_state=42)
mlp.fit(X_train_scaled, y_train)

# 在测试集上评估模型
y_pred = mlp.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nBP神经网络在处理缺失值后的测试准确率: {accuracy:.2f}")

print("\n分类报告:")
print(classification_report(y_test, y_pred))

数据集中的缺失值:
feature1    95
feature2    93
label        0
dtype: int64

BP神经网络无法直接处理缺失值:
Input X contains NaN.
MLPClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

缺失值已被均值填充

BP神经网络在处理缺失值后的测试准确率: 0.46

分类报告:
              precision    recall  f1-score   support

           0       0.48      0.58      0.52       102
           1       0.43      0.34      0.38        98

    accuracy                           0.46       200
   macro avg   